# Lesson 2: Protecting Routes with Middleware

# Protecting Routes with Middleware

Welcome to the next step in building your full-featured To-Do list application! Now that you have learned how to implement user authentication, it's crucial to ensure that only authenticated users can access specific parts of your application. In this unit, we will explore how to protect routes using middleware in Django.

## What You'll Learn
In this section, you'll discover how to implement middleware to secure your application routes. We'll cover the following key points:

- **Understanding Middleware**: Middleware acts as a bridge between the web server and the web application. It processes requests before they reach the view and can modify responses before sending them to the client.
- **Creating and Using Middleware**: You'll learn how to create custom middleware to check for user authentication.

### Example: Custom Middleware

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        # This returns the view function for the current route, e.g., 'www.example.com/login' resolves to 'login'
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if request.headers.get('Authorization') != 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)
```

### Code Breakdown:
1. **Unprotected Routes**: The `AuthMiddleware` class checks if the current route is in the `unprotected_routes` list. If it is, the middleware allows the request to proceed. This is useful for routes like login, register, and logout, which should be accessible to all users, regardless of authentication status.
2. **Authentication Check**: If the route is not in `unprotected_routes`, the middleware checks for a valid `Authorization` header. If missing or incorrect, it returns a `401 Unauthorized` response. (In real-world scenarios, tokens would be validated against a database rather than hardcoded.)
3. **Middleware Chain**: The middleware either calls the next middleware in the chain or proceeds to the view if there are no further middlewares.

### Applying Middleware in Django Settings

To protect your views with the custom middleware, add it to your Django settings:

```python
MIDDLEWARE = [
    # ... other middleware
    'myapp.middleware.AuthMiddleware',
]
```

## Returning Token in Login Response

To authenticate users, you need to return a token when they log in. This token can be used to authorize future requests. Here's an example of how to return a token in the login response:

```python
from django.http import JsonResponse
from django.contrib.auth import login, authenticate
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        # Validate username and password (not shown here)
        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the token in the response (for simplicity, we use a fixed token value)
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})
```

In this example, the token `'abc123'` is returned for demonstration purposes. In a real-world application, you would generate a unique token for each user and store it securely.

## Sending Token in Subsequent Requests

After logging in, you can send the token in the `Authorization` header for future requests:

```python
import requests

url = 'http://localhost:3000/protected-route'
headers = {'Authorization': 'Token abc123'}
response = requests.get(url, headers=headers)
```

In this example, a GET request is sent to `protected-route` with the token `'abc123'` in the `Authorization` header. This token is validated by the `AuthMiddleware` we created earlier.

## Why It Matters

Middleware is an essential part of web security because it:
- **Adds an Extra Layer of Security**: It checks each request before it reaches the view, ensuring only authorized users can access restricted parts of your application.
- **Centralizes Control**: Instead of adding checks in every view, you can handle access control in one place, making your code cleaner and easier to maintain.
- **Enhances User Experience**: By ensuring users are appropriately authenticated, you provide a smoother and more secure experience, preventing unauthorized access seamlessly.

Securing your application with middleware not only protects sensitive data but also adds professionalism and reliability to your app. Ready to dive into practice? Let's get started!

## Protecting Routes with Middleware

To observe the behavior of the `AuthMiddleware` that restricts access to protected routes, follow the steps described below:

1. **Middleware Overview**: The `AuthMiddleware` checks if a request is targeting a protected route and validates the `Authorization` token in the headers. If the correct token isn't provided, it returns a `401 Access Denied` response.

2. **Code Overview**:
    - The middleware protects any route not listed in `unprotected_routes` (e.g., 'login', 'register', 'logout'). 
    - If the `Authorization` header does not contain a valid token (in this example, `'Token abc123'`), access is denied.

### Running the Code:
1. **Register User**:
    - A new user is registered by sending a POST request to the `/register/` endpoint.
    - The response will confirm whether the registration was successful.

2. **Login User**:
    - The user logs in, and upon successful authentication, the server returns a fixed CSRF token (`'abc123'`).
    - This token is used for authentication in protected routes.

3. **Access Protected Route with Valid Token**:
    - A GET request is sent to the `/protected/` route, including the correct token (`'abc123'`) in the `Authorization` header.
    - Since the token is valid, the response should return `200 OK` along with the message `'This is a protected route'`.

4. **Access Protected Route with Invalid Token**:
    - A similar GET request is sent to the `/protected/` route, but this time the token is invalid (`'Token invalid_token'`).
    - The middleware will block the request, returning a `401 Access Denied` response.

5. **Logout User**:
    - Finally, the user logs out, and a success message is returned upon successful logout.

### Sample Output:

1. **Register Response**:
    ```
    Register Response: 201 - {'message': 'User registered successfully'}
    ```

2. **Login Response**:
    ```
    Login Response: 200 - {'message': 'User logged in successfully', 'csrf_token': 'abc123'}
    ```

3. **Access Protected Route with Valid Token**:
    ```
    Protected Route Response: 200 - {'message': 'This is a protected route'}
    ```

4. **Access Protected Route with Invalid Token**:
    ```
    Protected Route Response: 401 - {'message': 'Access denied'}
    ```

5. **Logout Response**:
    ```
    Logout Response: 200 - {'message': 'User logged out successfully'}
    ```

### Conclusion:
The middleware effectively protects the `/protected/` route by checking for a valid `Authorization` token in the request headers. If the token is missing or incorrect, the middleware prevents access, ensuring that only authenticated users with the correct token can access the protected parts of the application.

Nice work on understanding how middleware works! Now, let's put your knowledge into action by running some code and observing its output.

In this task, we have new AuthMiddleware that restricts access to protected routes when the correct token is not provided. The middleware checks if the current route is protected and if the correct token is provided in the request headers. If the token is not provided or is incorrect, the middleware returns a 401 status code with the message "Access denied".

Run the code and see how the middleware restricts access to protected routes when the correct token is not provided.

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)
        
        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)



from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@login_required
def protected_view(request):
    return JsonResponse({'message': 'This is a protected route'})



import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Request the protected route with valid token
headers = {
    'Authorization': f'Token {csrf_token}'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Request the protected route with invalid token
headers = {
    'Authorization': f'Token invalid_token'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")



from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout, protected_view

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
    path('protected/', protected_view, name='protected_view'),
]

```

## Protect Important Routes with Middleware

Building on our understanding of middleware, it's time to make a small adjustment.

In the AuthMiddleware class, we have a list of unprotected routes that are allowed to be accessed without a token. We need to modify the logic, to make the dashboard route protected as well.

Can you analyze the code and make the necessary changes to the AuthMiddleware class to ensure that the dashboard route is protected and requires a valid token to access it?

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout', 'dashboard']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)

from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@login_required
def dashboard(request):
    return JsonResponse({'message': 'This is a protected route'})

import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Request the protected route with valid token
headers = {
    'Authorization': f'Token {csrf_token}'
}
response = session.get(BASE_URL + 'dashboard/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Request the protected route with invalid token
headers = {
    'Authorization': f'Token invalid_token'
}
response = session.get(BASE_URL + 'dashboard/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")

from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout, dashboard

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
    path('dashboard/', dashboard, name='dashboard'),
]```

To make the `dashboard` route protected in your Django app, you need to modify the `AuthMiddleware` class. The current logic mistakenly includes `'dashboard'` in the `unprotected_routes`, meaning that it is accessible without a token. To fix this, simply remove `'dashboard'` from the `unprotected_routes` list.

Here's the updated `AuthMiddleware`:

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        # 'dashboard' has been removed from unprotected routes, making it protected
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)

        return self.get_response(request)
```

### Changes Made:
- **Removed `'dashboard'` from `unprotected_routes`:**
  Now, access to the `/dashboard/` route is protected and requires a valid token in the `Authorization` header.

### How it Works:
1. **Unprotected Routes:** Routes like `login`, `register`, and `logout` are accessible without authentication.
2. **Protected Routes:** Now, `/dashboard/` is protected. It will require the correct token in the request header to allow access.

### Testing:

1. **Accessing the Protected Dashboard Route:**
   With a valid token (`Token abc123`), the middleware will allow access to the `/dashboard/` route:
   ```python
   # Request the protected route with valid token
   headers = {
       'Authorization': f'Token {csrf_token}'
   }
   response = session.get(BASE_URL + 'dashboard/', headers=headers)
   print(f"Protected Route Response: {response.status_code} - {response.json()}")
   ```

2. **Accessing with an Invalid Token:**
   If the token is invalid, the middleware will return a `401 Access Denied`:
   ```python
   # Request the protected route with invalid token
   headers = {
       'Authorization': 'Token invalid_token'
   }
   response = session.get(BASE_URL + 'dashboard/', headers=headers)
   print(f"Protected Route Response: {response.status_code} - {response.json()}")
   ```

Now, your `/dashboard/` route is correctly protected, requiring a valid token for access.

## Fix the Middleware Bug

## Protect Routes with Middleware

## Sending Requests with Token